In [1]:
import os
import pandas as pd
import xarray as xr
import numpy as np
import pyvista as pv
pv.set_jupyter_backend('pythreejs')

Check if there is a directory with the Slab2.0 model

In [2]:
pwd = os.path.join("src")
if not os.path.exists(pwd):
    print('Error: Could not find source directory to Slab2.0.')

dep_file = os.path.join(pwd,'sam_slab2_dep_02.23.18.xyz')
thk_file = os.path.join(pwd,'sam_slab2_thk_02.23.18.xyz')
str_file = os.path.join(pwd,'sam_slab2_str_02.23.18.xyz')
dip_file = os.path.join(pwd,'sam_slab2_dip_02.23.18.xyz')

Read the top surface depth of the slab

In [3]:
data = pd.read_csv(dep_file, names=['lon', 'lat', 'dep'])
dep = data.pivot_table(index='lon', columns='lat', values='dep', dropna=False).T.values
dep = xr.DataArray(1.0E3 * dep, coords=[np.unique(data['lat']), np.unique(data['lon'])], dims=['lat', 'lon'])
dep.attrs['units'] = 'm'
dep = dep.to_dataset(name='dep')

Read the thickness of the slab

In [4]:
data = pd.read_csv(thk_file, names=['lon', 'lat', 'thk'])
thk = data.pivot_table(index='lon', columns='lat', values='thk', dropna=False).T.values
thk = xr.DataArray(1.0E3 * thk, coords=[np.unique(data['lat']), np.unique(data['lon'])], dims=['lat', 'lon'])
thk.attrs['units'] = 'm'
thk = thk.to_dataset(name='thk')

Read the strike angle of the slab

In [5]:
data = pd.read_csv(str_file, names=['lon', 'lat', 'str'])
strike = data.pivot_table(index='lon', columns='lat', values='str', dropna=False).T.values
strike *= np.pi / 180.
strike = xr.DataArray(strike, coords=[np.unique(data['lat']), np.unique(data['lon'])], dims=['lat', 'lon'])
strike.attrs['units'] = 'rad'
strike = strike.to_dataset(name='str')

Read the dip angle of the slab

In [6]:
data = pd.read_csv(dip_file, names=['lon', 'lat', 'dip'])
dip = data.pivot_table(index='lon', columns='lat', values='dip', dropna=False).T.values
dip *= np.pi / 180.
dip = xr.DataArray(dip, coords=[np.unique(data['lat']), np.unique(data['lon'])], dims=['lat', 'lon'])
dip.attrs['units'] = 'rad'
dip = dip.to_dataset(name='dip')

Merge DataArrays into a Dataset

In [7]:
slab2 = xr.merge([dep, thk, strike, dip])

In [8]:
X = slab2.coords['lon']
Y = slab2.coords['lat']
X, Y = np.meshgrid(X, Y)
Z = slab2.dep.values / 1_000.0 / 100

mesh = pv.StructuredGrid(X, Y, Z)
mesh["dep"] = Z.flatten(order="F")

In [9]:
plotter = pv.Plotter(multi_samples=8)
plotter.add_mesh(mesh)
plotter.add_bounding_box()
plotter.camera_position='xy'
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…